In [1]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import pandas as pd
import scipy
import scipy.optimize as opt
import seaborn as sns
import statsmodels.api as sm
from numpy.linalg import inv
from pypolyagamma import PyPolyaGamma
from scipy.stats import multivariate_normal
from tqdm import tqdm

from bayesian_statistics.utils_2 import *

In [ ]:
def create_complex_design_matrix(
    xy: np.ndarray, omega_x=np.pi / 5, omega_y=np.pi / 5, omega_xy=np.pi / 10
) -> np.ndarray:
    """2次元の複雑な設計行列を生成する関数。xy の形状は (N, 2), 返り値は(N, p)"""
    x = xy[:, 0]
    y = xy[:, 1]
    return np.column_stack(
        [
            np.ones_like(x),  # バイアス項
            x,  # x の一次項
            y,  # y の一次項
            x**2,  # x の二次項
            y**2,  # y の二次項
            x * y,  # 相互作用項
            np.sin(omega_x * x),  # x に関する正弦関数
            np.cos(omega_x * x),  # x に関する余弦関数
            np.sin(omega_y * y),  # y に関する正弦関数
            np.cos(omega_y * y),  # y に関する余弦関数
            np.sin(omega_xy * (x + y)),  # x + y に関する正弦関数
            np.cos(omega_xy * (x - y)),  # x - y に関する余弦関数
        ]
    )


# 真のパラメータ
beta_true = np.array(
    [
        -1.0,  # バイアス項
        -0.1,  # x の一次項
        -0.5,  # y の一次項
        -0.05,  # x^2 の項
        -0.1,  # y^2 の項
        0.2,  # x * y の項
        4.0,  # sin(ωx x) の項
        -4.0,  # cos(ωx x) の項
        4.0,  # sin(ωy y) の項
        -4.0,  # cos(ωy y) の項
        -9.0,  # sin(ωxy (x + y)) の項
        -7.0,  # cos(ωxy (x - y)) の項
    ]
)
lambda_star_true = 1.0  # 真の λ*


# 領域の設定
region = [(0, 20), (0, 20)]  # x と y の範囲

# 真の強度関数の設定
true_intensity_func = IntensityFunction(
    design_matrix_func=create_complex_design_matrix,
    beta=beta_true,
    lambda_star=lambda_star_true,
)

# データの生成
W_obs, y_obs, X_obs = generate_IPP(region, true_intensity_func, generate_U_flag=False)

# イベントのプロット
plot_events_2d(region, X_obs[y_obs == 1], true_intensity_func, plot_intensity=False)
plot_events_2d(region, X_obs[y_obs == 1], true_intensity_func, plot_intensity=True)

In [ ]:
create_complex_design_matrix(X_obs[y_obs == 1]).shape

In [ ]:
X_obs.shape

In [ ]:
# 事前分布の設定
prior_beta_mean = np.zeros(len(beta_true))
prior_beta_cov = np.eye(len(beta_true)) * 10
prior_lambda_shape = 2.0
prior_lambda_rate = 1.0

beta_init = np.random.multivariate_normal(prior_beta_mean, prior_beta_cov)
lambda_star_init = np.random.gamma(prior_lambda_shape, 1 / prior_lambda_rate)

intensity_func = IntensityFunction(
    create_complex_design_matrix, beta_init, lambda_star_init
)

# MCMC の実行
num_iterations = 10000
beta_samples, lambda_star_samples = mcmc_sampler(
    region=region,
    intensity_func=intensity_func,
    y_obs=y_obs,
    X_obs=X_obs,
    num_iterations=num_iterations,
    prior_beta_mean=prior_beta_mean,
    prior_beta_cov=prior_beta_cov,
    prior_lambda_shape=prior_lambda_shape,
    prior_lambda_rate=prior_lambda_rate,
)

burn_in = 500
beta_samples = np.array(beta_samples)[burn_in:, :]
lambda_star_samples = np.array(lambda_star_samples)[burn_in:]

In [ ]:
import arviz as az

# サンプル数とパラメータ数の取得
num_samples, num_beta = beta_samples.shape

# パラメータ名とサンプルの辞書を作成
posterior_samples = {}

# βの各成分を辞書に追加
for i in range(num_beta):
    param_name = f"beta_{i}"
    posterior_samples[param_name] = beta_samples[:, i]

# λ*を辞書に追加
posterior_samples["lambda_star"] = lambda_star_samples

# InferenceDataオブジェクトを作成
idata = az.from_dict(posterior=posterior_samples)

# --- ArviZによる可視化と解析 ---
az.style.use("arviz-doc")


# トレースプロット
az.plot_trace(idata)
plt.show()

# 事後分布のプロット
az.plot_posterior(idata)
plt.show()

# 自己相関プロット
az.plot_autocorr(idata)
plt.show()

# 統計的な概要情報の表示
summary_df = az.summary(idata)
display(summary_df)

# スタイルをリセット
az.style.use("default")

In [ ]:
plot_intensity_posterior_2d(
    region=region,
    beta_samples=np.array(beta_samples),
    lambda_star_samples=np.array(lambda_star_samples),
    intensity_func=true_intensity_func.copy(),
    num_points=100,
    credible_interval=0.95,
    true_intensity_func=true_intensity_func.lambda_func,
    events=X_obs[y_obs == 1],
)

ほとんど強度関数は復元できているが、強度の最大値が過小評価されている気がする。